In [5]:
import yfinance as yf
import pandas as pd

# Define ticker
ticker = yf.Ticker("AAPL")  # Change to any ticker you want

# Fetch financial data
income_stmt = ticker.financials.T  # Transpose so years are rows
cash_flow = ticker.cashflow.T
balance_sheet = ticker.balance_sheet.T
info = ticker.info  # Dictionary of metadata

# Filter last 4 years
income_stmt = income_stmt.head(4)
cash_flow = cash_flow.head(4)
balance_sheet = balance_sheet.head(4)

# Extract relevant fields (safely)
income_fields = [
    'Total Revenue',
    'Operating Income',
    'Income Before Tax',
    'Income Tax Expense'
]
income_available = [col for col in income_fields if col in income_stmt.columns]
income_df = income_stmt[income_available].copy()

cashflow_fields = [
    'Operating Cash Flow',
    'Capital Expenditure'
]
cashflow_available = [col for col in cashflow_fields if col in cash_flow.columns]
cashflow_df = cash_flow[cashflow_available].copy()

balance_fields = [
    'Total Current Assets',
    'Total Current Liabilities',
    'Cash And Cash Equivalents'
]
balance_available = [col for col in balance_fields if col in balance_sheet.columns]
balance_df = balance_sheet[balance_available].copy()

# Stock info (Beta, Market Cap, etc.)
info_keys = ['beta', 'marketCap', 'sharesOutstanding', 'sector']
info_clean = {key: info[key] for key in info_keys if key in info}
info_df = pd.DataFrame.from_dict(info_clean, orient='index', columns=['Value'])

# Optional: round values for readability
income_df = income_df.round(0)
cashflow_df = cashflow_df.round(0)
balance_df = balance_df.round(0)

# Display
print("📄 Income Statement (Last 4 Years):\n", income_df, "\n")
print("💵 Cash Flow Statement (Last 4 Years):\n", cashflow_df, "\n")
print("📊 Balance Sheet (Last 4 Years):\n", balance_df, "\n")
print("📈 Stock Info:\n", info_df)

📄 Income Statement (Last 4 Years):
             Total Revenue  Operating Income
2024-09-30   3.910350e+11      1.232160e+11
2023-09-30   3.832850e+11      1.143010e+11
2022-09-30   3.943280e+11      1.194370e+11
2021-09-30   3.658170e+11      1.089490e+11 

💵 Cash Flow Statement (Last 4 Years):
             Operating Cash Flow  Capital Expenditure
2024-09-30         1.182540e+11        -9.447000e+09
2023-09-30         1.105430e+11        -1.095900e+10
2022-09-30         1.221510e+11        -1.070800e+10
2021-09-30         1.040380e+11        -1.108500e+10 

📊 Balance Sheet (Last 4 Years):
             Cash And Cash Equivalents
2024-09-30               2.994300e+10
2023-09-30               2.996500e+10
2022-09-30               2.364600e+10
2021-09-30               3.494000e+10 

📈 Stock Info:
                            Value
beta                       1.165
marketCap          3014019776512
sharesOutstanding    14840399872
sector                Technology


In [6]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# --------- Config ---------
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']  # Replace with your 1000 tickers
YEARS = 4

# --------- Core Extraction Function ---------

def get_financial_data(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        
        # Download statements
        income_stmt = ticker.financials.T.head(YEARS)
        cash_flow = ticker.cashflow.T.head(YEARS)
        balance_sheet = ticker.balance_sheet.T.head(YEARS)
        info = ticker.info

        # Income Statement
        income_fields = ['Total Revenue', 'Operating Income', 'Income Before Tax', 'Income Tax Expense']
        income_df = income_stmt[[col for col in income_fields if col in income_stmt.columns]].copy()
        income_df['Ticker'] = ticker_symbol

        # Cash Flow Statement
        cashflow_fields = ['Operating Cash Flow', 'Capital Expenditure']
        cashflow_df = cash_flow[[col for col in cashflow_fields if col in cash_flow.columns]].copy()
        cashflow_df['Ticker'] = ticker_symbol

        # Balance Sheet
        balance_fields = ['Total Current Assets', 'Total Current Liabilities', 'Cash And Cash Equivalents']
        balance_df = balance_sheet[[col for col in balance_fields if col in balance_sheet.columns]].copy()
        balance_df['Ticker'] = ticker_symbol

        # Stock Info
        info_fields = ['beta', 'marketCap', 'sharesOutstanding', 'sector']
        info_clean = {key: info.get(key, None) for key in info_fields}
        info_clean['Ticker'] = ticker_symbol
        info_df = pd.DataFrame([info_clean])
        
        return income_df, cashflow_df, balance_df, info_df

    except Exception as e:
        print(f"❌ Error with {ticker_symbol}: {e}")
        return None, None, None, None

# --------- Loop Through Tickers ---------

all_income = []
all_cashflow = []
all_balance = []
all_info = []

for t in tqdm(TICKERS):
    income, cashflow, balance, info = get_financial_data(t)
    
    if income is not None:
        all_income.append(income)
        all_cashflow.append(cashflow)
        all_balance.append(balance)
        all_info.append(info)

# --------- Concatenate Results ---------

df_income = pd.concat(all_income).reset_index().rename(columns={'index': 'Date'})
df_cashflow = pd.concat(all_cashflow).reset_index().rename(columns={'index': 'Date'})
df_balance = pd.concat(all_balance).reset_index().rename(columns={'index': 'Date'})
df_info = pd.concat(all_info).reset_index(drop=True)

# Optional: round values for readability
df_income = df_income.round(0)
df_cashflow = df_cashflow.round(0)
df_balance = df_balance.round(0)

# --------- Done ---------
print("✅ Completed extraction.")

# If needed: Save to files
# df_income.to_csv('income.csv', index=False)
# df_cashflow.to_csv('cashflow.csv', index=False)
# df_balance.to_csv('balance.csv', index=False)
# df_info.to_csv('stock_info.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.25s/it]

✅ Completed extraction.


In [17]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# --------- Config ---------
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']  # Replace with your 1000 tickers
YEARS = 4

# --------- Corrected Fields ---------
income_fields = ['Total Revenue', 'Operating Income', 'Pretax Income', 'Tax Provision']
cashflow_fields = ['Operating Cash Flow', 'Capital Expenditure', 'Free Cash Flow', 
                   'Depreciation And Amortization', 'Stock Based Compensation']
balance_fields = ['Current Assets', 'Current Liabilities', 'Cash And Cash Equivalents']

info_fields = ['beta', 'marketCap', 'sharesOutstanding', 'sector']

# --------- Logging lists ---------
missing_log = []

# --------- Core Extraction Function ---------

def get_financial_data(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)

        # Fetch financials
        income_stmt = ticker.financials.T.head(YEARS)
        cash_flow = ticker.cashflow.T.head(YEARS)
        balance_sheet = ticker.balance_sheet.T.head(YEARS)
        info = ticker.info

        # --- Income Statement ---
        for col in income_fields:
            if col not in income_stmt.columns:
                income_stmt[col] = pd.NA
                missing_log.append((ticker_symbol, 'Income Statement', col))
        income_df = income_stmt[income_fields].copy()
        income_df['Ticker'] = ticker_symbol

        # --- Cash Flow Statement ---
        for col in cashflow_fields:
            if col not in cash_flow.columns:
                cash_flow[col] = pd.NA
                missing_log.append((ticker_symbol, 'Cash Flow', col))
        cashflow_df = cash_flow[cashflow_fields].copy()
        cashflow_df['Ticker'] = ticker_symbol

        # --- Balance Sheet ---
        for col in balance_fields:
            if col not in balance_sheet.columns:
                balance_sheet[col] = pd.NA
                missing_log.append((ticker_symbol, 'Balance Sheet', col))
        balance_df = balance_sheet[balance_fields].copy()
        balance_df['Ticker'] = ticker_symbol

        # --- Info ---
        info_clean = {key: info.get(key, None) for key in info_fields}
        info_clean['Ticker'] = ticker_symbol
        info_df = pd.DataFrame([info_clean])

        return income_df, cashflow_df, balance_df, info_df

    except Exception as e:
        print(f"❌ Error with {ticker_symbol}: {e}")
        return None, None, None, None

# --------- Loop Through Tickers ---------

all_income = []
all_cashflow = []
all_balance = []
all_info = []

for t in tqdm(TICKERS):
    income, cashflow, balance, info = get_financial_data(t)
    if income is not None:
        all_income.append(income)
        all_cashflow.append(cashflow)
        all_balance.append(balance)
        all_info.append(info)

# --------- Concatenate Results ---------

df_income = pd.concat(all_income).reset_index().rename(columns={'index': 'Date'})
df_cashflow = pd.concat(all_cashflow).reset_index().rename(columns={'index': 'Date'})
df_balance = pd.concat(all_balance).reset_index().rename(columns={'index': 'Date'})
df_info = pd.concat(all_info).reset_index(drop=True)

# --------- Cleanup ---------
df_income['Date'] = pd.to_datetime(df_income['Date'])
df_cashflow['Date'] = pd.to_datetime(df_cashflow['Date'])
df_balance['Date'] = pd.to_datetime(df_balance['Date'])

df_income = df_income.round(0)
df_cashflow = df_cashflow.round(0)
df_balance = df_balance.round(0)



pd.DataFrame(missing_log, columns=['Ticker', 'Statement', 'Missing Column']).to_csv('missing_fields_log.csv', index=False)

print("✅ Done. Missing fields logged to 'missing_fields_log.csv'")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.02it/s]

✅ Done. Missing fields logged to 'missing_fields_log.csv'


In [18]:
df_income

Date  Total Revenue  Operating Income  Pretax Income  Tax Provision  \
0  2024-09-30   3.910350e+11      1.232160e+11   1.234850e+11   2.974900e+10   
1  2023-09-30   3.832850e+11      1.143010e+11   1.137360e+11   1.674100e+10   
2  2022-09-30   3.943280e+11      1.194370e+11   1.191030e+11   1.930000e+10   
3  2021-09-30   3.658170e+11      1.089490e+11   1.092070e+11   1.452700e+10   
4  2025-06-30   2.817240e+11      1.285280e+11   1.236270e+11   2.179500e+10   
5  2024-06-30   2.451220e+11      1.094330e+11   1.077870e+11   1.965100e+10   
6  2023-06-30   2.119150e+11      8.852300e+10   8.931100e+10   1.695000e+10   
7  2022-06-30   1.982700e+11      8.338300e+10   8.371600e+10   1.097800e+10   
8  2024-12-31   3.500180e+11      1.123900e+11   1.198150e+11   1.969700e+10   
9  2023-12-31   3.073940e+11      8.429300e+10   8.571700e+10   1.192200e+10   
10 2022-12-31   2.828360e+11      7.484200e+10   7.132800e+10   1.135600e+10   
11 2021-12-31   2.576370e+11      7.871400e+10   9.073400e+10   1.470100e+10   
12 2024-12-31   6.379590e+11      6.859300e+10   6.861400e+10   9.265000e+09   
13 2023-12-31   5.747850e+11      3.685200e+10   3.755700e+10   7.120000e+09   
14 2022-12-31   5.139830e+11      1.224800e+10  -5.936000e+09  -3.217000e+09   
15 2021-12-31   4.698220e+11      2.487900e+10   3.815100e+10   4.791000e+09   
16 2024-12-31   1.645010e+11      6.938000e+10   7.066300e+10   8.303000e+09   
17 2023-12-31   1.349020e+11      4.675100e+10   4.742800e+10   8.330000e+09   
18 2022-12-31   1.166090e+11      2.894400e+10   2.881900e+10   5.619000e+09   
19 2021-12-31   1.179290e+11      4.675300e+10   4.728400e+10   7.914000e+09   

   Ticker  
0    AAPL  
1    AAPL  
2    AAPL  
3    AAPL  
4    MSFT  
5    MSFT  
6    MSFT  
7    MSFT  
8   GOOGL  
9   GOOGL  
10  GOOGL  
11  GOOGL  
12   AMZN  
13   AMZN  
14   AMZN  
15   AMZN  
16   META  
17   META  
18   META  
19   META

In [19]:
df_cashflow

Date  Operating Cash Flow  Capital Expenditure  Free Cash Flow  \
0  2024-09-30         1.182540e+11        -9.447000e+09    1.088070e+11   
1  2023-09-30         1.105430e+11        -1.095900e+10    9.958400e+10   
2  2022-09-30         1.221510e+11        -1.070800e+10    1.114430e+11   
3  2021-09-30         1.040380e+11        -1.108500e+10    9.295300e+10   
4  2025-06-30         1.361620e+11        -6.455100e+10    7.161100e+10   
5  2024-06-30         1.185480e+11        -4.447700e+10    7.407100e+10   
6  2023-06-30         8.758200e+10        -2.810700e+10    5.947500e+10   
7  2022-06-30         8.903500e+10        -2.388600e+10    6.514900e+10   
8  2024-12-31         1.252990e+11        -5.253500e+10    7.276400e+10   
9  2023-12-31         1.017460e+11        -3.225100e+10    6.949500e+10   
10 2022-12-31         9.149500e+10        -3.148500e+10    6.001000e+10   
11 2021-12-31         9.165200e+10        -2.464000e+10    6.701200e+10   
12 2024-12-31         1.158770e+11        -8.299900e+10    3.287800e+10   
13 2023-12-31         8.494600e+10        -5.272900e+10    3.221700e+10   
14 2022-12-31         4.675200e+10        -6.364500e+10   -1.689300e+10   
15 2021-12-31         4.632700e+10        -6.105300e+10   -1.472600e+10   
16 2024-12-31         9.132800e+10        -3.725600e+10    5.407200e+10   
17 2023-12-31         7.111300e+10        -2.704500e+10    4.406800e+10   
18 2022-12-31         5.047500e+10        -3.118600e+10    1.928900e+10   
19 2021-12-31         5.768300e+10        -1.869000e+10    3.899300e+10   

    Depreciation And Amortization  Stock Based Compensation Ticker  
0                    1.144500e+10              1.168800e+10   AAPL  
1                    1.151900e+10              1.083300e+10   AAPL  
2                    1.110400e+10              9.038000e+09   AAPL  
3                    1.128400e+10              7.906000e+09   AAPL  
4                    3.415300e+10              1.197400e+10   MSFT  
5                    2.228700e+10              1.073400e+10   MSFT  
6                    1.386100e+10              9.611000e+09   MSFT  
7                    1.446000e+10              7.502000e+09   MSFT  
8                    1.531100e+10              2.278500e+10  GOOGL  
9                    1.194600e+10              2.246000e+10  GOOGL  
10                   1.347500e+10              1.936200e+10  GOOGL  
11                   1.244100e+10              1.537600e+10  GOOGL  
12                   5.279500e+10              2.201100e+10   AMZN  
13                   4.866300e+10              2.402300e+10   AMZN  
14                   4.192100e+10              1.962100e+10   AMZN  
15                   3.443300e+10              1.275700e+10   AMZN  
16                   1.549800e+10              1.669000e+10   META  
17                   1.117800e+10              1.402700e+10   META  
18                   8.686000e+09              1.199200e+10   META  
19                   7.967000e+09              9.164000e+09   META

In [20]:
df_balance

Date  Current Assets  Current Liabilities  Cash And Cash Equivalents  \
0  2024-09-30    1.529870e+11         1.763920e+11               2.994300e+10   
1  2023-09-30    1.435660e+11         1.453080e+11               2.996500e+10   
2  2022-09-30    1.354050e+11         1.539820e+11               2.364600e+10   
3  2021-09-30    1.348360e+11         1.254810e+11               3.494000e+10   
4  2025-06-30    1.911310e+11         1.412180e+11               3.024200e+10   
5  2024-06-30    1.597340e+11         1.252860e+11               1.831500e+10   
6  2023-06-30    1.842570e+11         1.041490e+11               3.470400e+10   
7  2022-06-30    1.696840e+11         9.508200e+10               1.393100e+10   
8  2024-12-31    1.637110e+11         8.912200e+10               2.346600e+10   
9  2023-12-31    1.715300e+11         8.181400e+10               2.404800e+10   
10 2022-12-31    1.647950e+11         6.930000e+10               2.187900e+10   
11 2021-12-31    1.881430e+11         6.425400e+10               2.094500e+10   
12 2024-12-31    1.908670e+11         1.794310e+11               7.877900e+10   
13 2023-12-31    1.723510e+11         1.649170e+11               7.338700e+10   
14 2022-12-31    1.467910e+11         1.553930e+11               5.388800e+10   
15 2021-12-31    1.615800e+11         1.422660e+11               3.622000e+10   
16 2024-12-31    1.000450e+11         3.359600e+10               4.388900e+10   
17 2023-12-31    8.536500e+10         3.196000e+10               4.186200e+10   
18 2022-12-31    5.954900e+10         2.702600e+10               1.468100e+10   
19 2021-12-31    6.666600e+10         2.113500e+10               1.660100e+10   

   Ticker  
0    AAPL  
1    AAPL  
2    AAPL  
3    AAPL  
4    MSFT  
5    MSFT  
6    MSFT  
7    MSFT  
8   GOOGL  
9   GOOGL  
10  GOOGL  
11  GOOGL  
12   AMZN  
13   AMZN  
14   AMZN  
15   AMZN  
16   META  
17   META  
18   META  
19   META

In [ ]:
# caluclate present FCFF, Use 4 year average to find growth rate for next three years
# Terminal value calcualtion after 3 years, I need to find the WACC? Estimate or ML? First estimate, ML as an add on later.